# **ML Classifier**

In [ ]:
import numpy as np

def splitdata_train_test(data, fraction_training)
  np.random.shuffle(data)
  split_index = int(fraction_training*len(data))
  return data[:split_index], data[split_index:]

  data = np.load('galaxy_catalogue.npy')
  fraction_training = 0.7

  training, testing = splitdata_train_test(data, fraction_training)

  print('Number data galaxies:', len(data))
  print('Train fraction:', fraction_training)
  print('Number of galaxies in training set:', len(training))
  print('Number of galaxies in testing set:', len(testing))

In [3]:
def generate_features_targets(data):
  targets = data['class']

  features = np.empty(shape=(len(data), 13))
  features[:, 0] = data['u-g']
  features[:, 1] = data['g-r']
  features[:, 2] = data['r-i']
  features[:, 3] = data['i-z']
  features[:, 4] = data['ecc']
  features[:, 5] = data['m4_u']
  features[:, 6] = data['m4_g']
  features[:, 7] = data['m4_r']
  features[:, 8] = data['m4_i']
  features[:, 9] = data['m4_z']

  features[:, 10] = data['petroR50_u']/data['petroR90_u']
  features[:, 11] = data['petroR50_r']/data['petroR90_r']
  features[:, 12] = data['petroR50_z']/data['petroR90_z']

  return features, targets

  data = np.load('galaxy_catalogue.npy')

  features, targets = generate_features_targets(data)
 
  print("Features shape:", features.shape)
  print("Targets shape:", targets.shape)

# Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier
def dtc_predict_actual(data):
  train, test = splitdata_train_test(data, 0.7)

  train_features, train_targets = generate_features_targets(train)
  test_features, test_targets = generate_features_targets(test)

  dtc = DecisionTreeClassifier()

  dtc.fit(train_features, train_targets)
  
  predictions = dtc.predict(test_features)
  return predictions, test_targets


  data = np.load('galaxy_catalogue.npy')
    
  predicted_class, actual_class = dtc_predict_actual(data)

  print("Some initial results...\n   predicted,  actual")
  for i in range(10):
    print("{}. {}, {}".format(i, predicted_class[i], actual_class[i]))
 

# Accuracy

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier


def calculate_accuracy(predicted_classes, actual_classes):
  return sum(predicted_classes == actual_classes)/len(actual_classes)

  data = np.load('galaxy_catalogue.npy')

  features, targets = generate_features_targets(data)

  dtc = DecisionTreeClassifier()
  predicted = cross_val_predict(dtc, features, targets, cv=10)

  model_score = calculate_accuracy(predicted, targets)
  print("Our accuracy score:", model_score)

  class_labels = list(set(targets))
  model_cm = confusion_matrix(y_true=targets, y_pred=predicted, labels=class_labels)

  plt.figure()
  plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
  plt.show()

# Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

def rf_predict_actual(data, n_estimators):

  features, targets = generate_features_targets(data)

  rfc = RandomForestClassifier(n_estimators=n_estimators)

  predicted = cross_val_predict(rfc, features, targets, cv=10)

  return predicted, targets

  data = np.load('galaxy_catalogue.npy')

  number_estimators = 50            
  predicted, actual = rf_predict_actual(data, number_estimators)

  accuracy = calculate_accuracy(predicted, actual)
  print("Accuracy score:", accuracy)

  class_labels = list(set(actual))
  model_cm = confusion_matrix(y_true=actual, y_pred=predicted, labels=class_labels)

  plt.figure()
  plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
  plt.show()